In [11]:
# On importe les packages utiles
import sqlalchemy as db
import pandas as pd

In [12]:
# On établit une connexion
engine = db.create_engine("sqlite:///burger.db")
connection = engine.connect()

In [29]:
metadata = db.MetaData()

pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True)
              )

### Gestion des restaurants
restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"), nullable=False),
              db.Column('capacité', db.Integer(), default=0),
              db.Column('espace_enfant', db.Integer(), default=False),
              db.Column('service_rapide', db.Integer(), default=False),
              db.Column('accessibilité', db.Integer(), default=False),
              db.Column('parking', db.Integer(), default=0)
              )

### Gestion des employés

employe_table = db.Table('Employe', metadata,
              db.Column('id_employe', db.Integer(), primary_key=True),
              db.Column('code_postal', db.String(), db.ForeignKey("Restaurant.code_postal"), nullable=False),
              db.Column('id_superieur', db.Integer(), db.ForeignKey("Employe.id_employe"), nullable=True),
              db.Column('poste', db.String(), nullable=False),
              db.Column('experience', db.Integer(), default=0, nullable=False),
              db.Column('nom', db.String(), nullable=False),
              db.Column('adresse', db.String(), nullable=False),
              db.Column('note', db.Integer(), default=1, nullable=False),
              )

rib_table = db.Table('RIB', metadata,
              db.Column('id_employe', db.Integer(), db.ForeignKey("Employe.id_employe"), primary_key=True),
              db.Column('iban', db.String(), nullable=False),
              db.Column('bic', db.String(), nullable=False),
              db.Column('proprietaire', db.String(), nullable=False),
              db.Column('adresse', db.String(), nullable=False),
              )

paie_table = db.Table('Paie', metadata,
              db.Column('id_employe', db.Integer(), db.ForeignKey("Employe.id_employe"), primary_key=True),
              db.Column('date', db.String(), primary_key=True),
              db.Column('salaire_net', db.Float(), nullable=False),
              )

### Gestion des cartes et menus

item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(), primary_key=True),
              db.Column('type', db.String(), nullable=False),
              db.Column('prix', db.Float(), nullable=False),
              )



menu_table = db.Table('Menu', metadata,
              db.Column('id_menu', db.Integer(), primary_key=True),
              db.Column('boisson', db.String(), db.ForeignKey("Item.nom_item"), nullable=False),
              db.Column('plat', db.String(), db.ForeignKey("Item.nom_item"), nullable=False),
              db.Column('dessert', db.String(), db.ForeignKey("Item.nom_item"), nullable=False),
              db.Column('prix', db.Float(), nullable=False),
              )


carte_menu_table = db.Table('CarteMenu', metadata,
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"), primary_key=True),
              db.Column('id_menu', db.Integer(), db.ForeignKey("Menu.id_menu"), primary_key=True),
              )

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"), primary_key=True),
              db.Column('nom_item', db.String(), db.ForeignKey("Item.nom_item"), primary_key=True),
              )

### Gestion des recettes et des stocks

ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('cout', db.Float(), nullable=False),
              )

recette_table = db.Table('Recette', metadata,
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"), primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"), primary_key=True),
              db.Column('quantite', db.Integer(),default=0)
              )

stock_table = db.Table('Stock', metadata,
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"), primary_key=True),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"), primary_key=True),
              db.Column('quantite', db.Integer(),default=0)
              )

# Gestion des tickets de caisse

bill_table = db.Table('Bill', metadata,
              db.Column('id_bill', db.Integer(), primary_key=True),
              db.Column('code_postal', db.Integer(), db.ForeignKey("Restaurant.code_postal"), nullable=False),
              db.Column('id_employe', db.Integer(), db.ForeignKey("Employe.id_employe"), nullable=True),
              db.Column('borne', db.Integer(), default=False, nullable=False),
              db.Column('moyen_paiement', db.String(), nullable=False),
              db.Column('prix_total', db.Float(), nullable=False)
              )

panier_menu_table = db.Table('PanierMenuTable', metadata,
              db.Column('id_bill', db.Integer(),db.ForeignKey("Bill.id_bill"), primary_key=True),
              db.Column('id_menu', db.Integer(),db.ForeignKey("Menu.id_menu"), primary_key=True),
              db.Column('quantite', db.Integer(),default=0)
              )

panier_item_table = db.Table('PanierItemTable', metadata,
              db.Column('id_bill', db.Integer(),db.ForeignKey("Bill.id_bill"), primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"), primary_key=True),
              db.Column('quantite', db.Integer(),default=0)
              )


In [30]:
# Attention, create_all est une méthode de l'objet MEtadata qui prend comme argument l'engine
metadata.create_all(engine) #Creates the table

# Modifier ma database

In [23]:
metadata_update = db.MetaData()

with engine.connect() as conn:
    metadata_update.reflect(conn)

In [25]:
metadata_update.tables["Menu"].drop(engine)